
<!-- 크롤링은? 
	데이터가 있다!! 
	토큰으로 쪼개서 영어로 변역해서 huggingface로 넣고 돌려 봐
	결과가 나와 그게 실제와 맞는지 확인해봐

	크롤링도 할 수 있음 해보는게 좋다. 
	 -->

In [ ]:
import pandas as pd 
import re
import tensorflow as tf

### 크롤링 나온 csv 토큰화 시키기

In [30]:
data = pd.read_csv("./NewsData/정치_20240704_10시18분.csv")
data.head()

,date,title,link,content
0,2024-07-04 05:36:12,"‘사퇴 없다’는 바이든, 지지율 급락에 정치 운명 초읽기",https://n.news.naver.com/mnews/article/005/000...,[\n\n\n\n\n조 바이든 대통령의 정치적 미래가 수일 내 결정될 수 있다는 관...
1,2024-07-03 18:14:31,검사 탄핵안 사유는 '정치적 중립 위반'…이해충돌 논란도,https://n.news.naver.com/mnews/article/001/001...,[\n한명숙·이화영·돈봉투 등 '민주당 수사' 검사들 표적수사·재판 당사자들이 탄핵...
2,2024-07-03 11:52:41,"인천 찾은 나경원 ""권력자 아닌 국민에 줄서는 정치해야""",https://n.news.naver.com/mnews/article/003/001...,"[\n“원희룡·한동훈 후보에 줄 서는 정치가 시작 돼""\n\n\n\n[인천=뉴시스]..."
3,2024-07-03 21:04:31,검찰총장 반발 놓고 “대놓고 정치” vs “이재명 수사가 죄인가”,https://n.news.naver.com/mnews/article/056/001...,[\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n [앵커]민주당의 검사 탄핵...
4,2024-07-03 12:02:48,"탄핵 100만 돌파…민주 ""정치사 유례 없어 필요하면 청문회""",https://n.news.naver.com/mnews/article/421/000...,"[\n강유정 ""불통과 독선 국정 반성하고 민심 받들어 국정 전환해야""\n\n\n\n..."


In [31]:
data['content'][0]

'[\n\n\n\n\n조 바이든 대통령의 정치적 미래가 수일 내 결정될 수 있다는 관측이 제기됐다. 바이든 대통령은 대선 캠프 직원들에게 레이스를 지속할 뜻을 직접 설명하며 후보직 사퇴론에 선을 그었지만, TV토론 영향이 여론조사에 반영되기 시작하면서 지지율 하락세가 분명해지고 있다. 조기에 분위기 반전 모멘텀을 잡지 못하면 상황은 걷잡을 수 없이 악화할 것이라는 우려가 당 안팎에서 제기됐다.바이든 대통령은 3일(현지시간) 민주당 전국위원회 전화 회의에 예고 없이 참석해 “민주당이 단결하면 우리는 항상 승리해 왔다. 이번에도 승리할 것”이라며 선거에 끝까지 참여하겠다는 메시지를 보냈다고 뉴욕타임스(NYT) 등 외신이 회의 참석자들을 인용해 보도했다.바이든 대통령은 “나는 민주당 리더이며 누구도 나를 밀어내지 못한다”며 “분명하고 명료하게 말하겠다. 나는 (대선에) 출마할 것”이라고 말했다. 또 “2020년에 도널드 트럼프를 이긴 것처럼 2024년에도 다시 이길 것”이라며 직원들에게 힘을 하나로 모아 달라고 부탁했다고 한다.카멀라 해리스 부통령도 직원들에게 “우리는 대통령 리더십을 따를 것”이라며 “계속 싸울 것이고 반드시 승리할 것”이라고 다독였다.바이든 대통령은 이날 해리스 부통령과 점심 식사를 함께하고, 하킴 제프리스 하원 원내대표와 척 슈머 상원 원내대표 등 당 지도부와 통화에 나섰다. 바이든 대통령은 이 자리에서도 선거 완주 의지를 강조하며 당내 혼란을 단속해 달라고 부탁한 것으로 알려졌다.캠프 측은 그러나 내부적으로는 바이든 대통령이 조기에 분위기를 반전해야 한다는 절박감에 사로잡혀 있다. 당 안팎에서 후보 교체론 논의가 계속 확산하고 있는 데다, 지지율 내림세도 뚜렷하기 때문이다.NYT는 “바이든 대통령은 재앙적이었던 지난주 TV 토론 이후 한 핵심 측근에게 ‘향후 며칠 안에 대통령직에 나설 수 있음을 대중들에게 납득시킬 수 없다면 대선 후보직을 구해낼 수 없을지도 모른다는 것을 알고 있다’고 말했다”고 보도하기도 했다. 백악관은 “완전한 거짓”이라

In [33]:
data2 = data.copy() # data2에 data 복제
data2.head()
data2.tail()

newContent = []

# \n, \t 불필요한 내용 제거를 위한 함수
def pad_punctuation(s):
    s = re.sub(r'\n', '', s)
    s = re.sub(r'\t', '', s)
    s = re.sub(r'\[', '', s)
    s = re.sub(r'\]', '', s)
    s = re.sub(' +', ' ', s)
    
    return s

# for문 돌리면서 기호 제거 
for i in data2['content'] :
    newContent.append(pad_punctuation(i))


# 일단은 df에 제거된 newContent리스트 추가 
data2['newContent'] = newContent
data2.head()

data2['newContent'][0]

'조 바이든 대통령의 정치적 미래가 수일 내 결정될 수 있다는 관측이 제기됐다. 바이든 대통령은 대선 캠프 직원들에게 레이스를 지속할 뜻을 직접 설명하며 후보직 사퇴론에 선을 그었지만, TV토론 영향이 여론조사에 반영되기 시작하면서 지지율 하락세가 분명해지고 있다. 조기에 분위기 반전 모멘텀을 잡지 못하면 상황은 걷잡을 수 없이 악화할 것이라는 우려가 당 안팎에서 제기됐다.바이든 대통령은 3일(현지시간) 민주당 전국위원회 전화 회의에 예고 없이 참석해 “민주당이 단결하면 우리는 항상 승리해 왔다. 이번에도 승리할 것”이라며 선거에 끝까지 참여하겠다는 메시지를 보냈다고 뉴욕타임스(NYT) 등 외신이 회의 참석자들을 인용해 보도했다.바이든 대통령은 “나는 민주당 리더이며 누구도 나를 밀어내지 못한다”며 “분명하고 명료하게 말하겠다. 나는 (대선에) 출마할 것”이라고 말했다. 또 “2020년에 도널드 트럼프를 이긴 것처럼 2024년에도 다시 이길 것”이라며 직원들에게 힘을 하나로 모아 달라고 부탁했다고 한다.카멀라 해리스 부통령도 직원들에게 “우리는 대통령 리더십을 따를 것”이라며 “계속 싸울 것이고 반드시 승리할 것”이라고 다독였다.바이든 대통령은 이날 해리스 부통령과 점심 식사를 함께하고, 하킴 제프리스 하원 원내대표와 척 슈머 상원 원내대표 등 당 지도부와 통화에 나섰다. 바이든 대통령은 이 자리에서도 선거 완주 의지를 강조하며 당내 혼란을 단속해 달라고 부탁한 것으로 알려졌다.캠프 측은 그러나 내부적으로는 바이든 대통령이 조기에 분위기를 반전해야 한다는 절박감에 사로잡혀 있다. 당 안팎에서 후보 교체론 논의가 계속 확산하고 있는 데다, 지지율 내림세도 뚜렷하기 때문이다.NYT는 “바이든 대통령은 재앙적이었던 지난주 TV 토론 이후 한 핵심 측근에게 ‘향후 며칠 안에 대통령직에 나설 수 있음을 대중들에게 납득시킬 수 없다면 대선 후보직을 구해낼 수 없을지도 모른다는 것을 알고 있다’고 말했다”고 보도하기도 했다. 백악관은 “완전한 거짓”이라고 강력히 부인했지만